In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# 1. Download historical data (example: RELIANCE.NS)
symbol = 'RELIANCE.NS'
start_date = '2023-01-01'
end_date = '2024-01-01'

data = yf.download(symbol, start=start_date, end=end_date)
data = data[['Open', 'High', 'Low', 'Close', 'Volume']]

# 2. Calculate EMAs for the strategy
data['EMA9'] = data['Close'].ewm(span=9, adjust=False).mean()
data['EMA21'] = data['Close'].ewm(span=21, adjust=False).mean()

# 3. Generate trading signals:
# Signal = 1 when EMA9 > EMA21 (bullish), else 0
# Position = 1 for buy signal, -1 for sell signal (crossover points)
data['Signal'] = 0
data.loc[9:, 'Signal'] = np.where(data['EMA9'][9:] > data['EMA21'][9:], 1, 0)
data['Position'] = data['Signal'].diff()

# 4. Initialize backtest variables & parameters

initial_cash = 100_000  # starting capital in INR
cash = initial_cash
position = 0            # 0 means no position, 1 means long position
n_shares = 0
entry_price = 0
trade_log = []

# Groww brokerage structure
def calculate_brokerage(trade_value):
    # brokerage is min(₹20, 0.1% of trade value) and min ₹5
    return max(5, min(20, trade_value * 0.001))

dp_charge_on_sell = 20  # ₹20 per sell transaction

# 5. Run through data and simulate trades

for i in range(1, len(data)):
    date = data.index[i]
    open_price = data['Open'].iloc[i]
    pos_signal = data['Position'].iloc[i]

    # Buy signal: crossover from 0 to 1
    if pos_signal == 1 and position == 0:
        # Calculate how many shares to buy (simple all-in)
        n_shares = cash // open_price
        if n_shares > 0:
            trade_value = n_shares * open_price
            brokerage = calculate_brokerage(trade_value)
            total_cost = trade_value + brokerage
            if total_cost <= cash:
                cash -= total_cost
                position = 1
                entry_price = open_price
                trade_log.append({
                    'Date': date,
                    'Type': 'Buy',
                    'Price': open_price,
                    'Shares': n_shares,
                    'Commission': brokerage,
                    'DP': 0,
                    'Cash': cash
                })

    # Sell signal: crossover from 1 to 0
    elif pos_signal == -1 and position == 1:
        trade_value = n_shares * open_price
        brokerage = calculate_brokerage(trade_value)
        dp_fee = dp_charge_on_sell
        proceeds = trade_value - brokerage - dp_fee
        cash += proceeds
        trade_log.append({
            'Date': date,
            'Type': 'Sell',
            'Price': open_price,
            'Shares': n_shares,
            'Commission': brokerage,
            'DP': dp_fee,
            'Cash': cash
        })
        position = 0
        n_shares = 0

# If position is still open, sell on last close price at end of data
if position == 1:
    last_date = data.index[-1]
    last_close = data['Close'].iloc[-1]
    trade_value = n_shares * last_close
    brokerage = calculate_brokerage(trade_value)
    dp_fee = dp_charge_on_sell
    proceeds = trade_value - brokerage - dp_fee
    cash += proceeds
    trade_log.append({
        'Date': last_date,
        'Type': 'Sell',
        'Price': last_close,
        'Shares': n_shares,
        'Commission': brokerage,
        'DP': dp_fee,
        'Cash': cash
    })

    position = 0
    n_shares = 0

# 6. Convert trade log to DataFrame for inspection
results = pd.DataFrame(trade_log)

# 7. Print basic performance stats
total_return = ((cash - initial_cash) / initial_cash) * 100
print(f"Backtest period: {start_date} to {end_date}")
print(f"Initial cash: ₹{initial_cash:,.2f}")
print(f"Final cash: ₹{cash:,.2f}")
print(f"Total return: {total_return:.2f}%")
print(f"Number of trades: {len(results)}")
print("\nTrade details:")
print(results)

# 8. Plot price and trades
plt.figure(figsize=(14, 7))
plt.plot(data['Close'], label='Close Price', color='blue')

# Plot buy signals (green triangles)
buys = results[results['Type'] == 'Buy']
plt.scatter(buys['Date'], buys['Price'], marker='^', color='green', s=100, label='Buy')

# Plot sell signals (red inverted triangles)
sells = results[results['Type'] == 'Sell']
plt.scatter(sells['Date'], sells['Price'], marker='v', color='red', s=100, label='Sell')

plt.title(f'Swing Trading EMA Crossover Backtest for {symbol}')
plt.xlabel('Date')
plt.ylabel('Price (₹)')
plt.legend()
plt.grid(True)
plt.show()


/var/folders/mn/tx231v5158797chnp22wm9000000gp/T/ipykernel_11949/730333092.py:11: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(symbol, start=start_date, end=end_date)
[*********************100%***********************]  1 of 1 completed


TypeError: Slicing a positional slice with .loc is not allowed, Use .loc with labels or .iloc with positions instead.

In [1]:
import yfinance as yf

stock = yf.Ticker("RELIANCE.NS")
info = stock.info

print("P/E Ratio:", info.get("trailingPE"))


P/E Ratio: 22.831978
